Imports


In [ ]:
from ultralytics import YOLO
# from trl import SFTTrainer, SFTConfig
# from peft import LoraConfig, get_peft_model
import warnings
import gc
import os
import re
import torch
# import pandas as pd
# import numpy as np

Disabling Warnings


In [2]:
warnings.filterwarnings("ignore")

Setting Training Device


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if str(device) == "cuda":
    print(f"Using GPU {torch.cuda.get_device_properties(device)}")
else:
    print("Using CPU")

Emptying the GPU Cache (if necessary)


In [ ]:
def empty_cache() -> None:
    # Cleaning out the device cache
    gc.collect()
    torch.cuda.empty_cache()


def print_free_memory():
    free, total = torch.cuda.mem_get_info(device)
    print(f"Percent of free memory: {round(free/total *100,2)}")


empty_cache()
print_free_memory()

Memory Summary


In [ ]:
def memory_summary() -> None:
    print(torch.cuda.memory_summary())


memory_summary()

Preparing GPU (if necessary)


In [ ]:
# For AMD GPU - 7800xt
device_name = torch.cuda.get_device_name(0)
if "AMD" in device_name or "Radeon" in device_name:
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = "11.0.0"

print(f"GPU {torch.cuda.get_device_properties(device).name} is now setup")

Setting Paths for Data, Base Model, and Output Directory


In [ ]:
# Directory Paths
training_set_name, data_method = "test_dataset", "clean", 
current_directory = os.getcwd()
path_to_base_directory = re.search(rf"(.*?){"Weird-Stuff-In-Traffic"}", current_directory).group(1)
yaml_data_path = f"Weird-Stuff-In-Traffic/Data/yolo/coco8/coco8.yaml"

# Model Paths
base_model_name = "yolo11n.pt"
simple_base_model_name = base_model_name.split(".")[0]
model_results_path = f"Weird-Stuff-In-Traffic/Models/Segmentation-Detection/yolo/"
model_training_config = f"training_{training_set_name}_{data_method}_data"
model_output_dir = path_to_base_directory + model_results_path + model_training_config + "/fine_tuned_model"

Loading Model


In [ ]:
model = YOLO(base_model_name)

Model Training


In [ ]:
# Train the model
model.train(
    data=yaml_data_path,
    epochs=1,
    imgsz=640,
    project=model_output_dir,
    name="experiment1",                # subfolder name for this run
    batch=16,                          # batch size (adjust for your GPU)
    workers=4,                         # number of data loading workers
    device=device,
    verbose=True
)